In [1]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from sentence_transformers import CrossEncoder
import dotenv
import os

dotenv.load_dotenv()

c:\anaconda3\envs\Modulabs\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

### 2-1. 문서 전처리

In [2]:
llm = ChatOpenAI(model="gpt-4o-mini",
                 temperature=0.,)

embeddings = OpenAIEmbeddings()

In [3]:
path = "./files/rules/"
filename = os.listdir(path)

In [4]:
# PDF 로더 생성

hr_loader = PyPDFLoader(path+filename[0])
security_loader = PyPDFLoader(path+filename[1])
onboard_loader = PyPDFLoader(path+filename[2])
tools_loader = PyPDFLoader(path+filename[3])
culture_loader = PyPDFLoader(path+filename[4])

In [5]:
# 텍스트 스플리터 생성

splitter = RecursiveCharacterTextSplitter(chunk_size=100, 
                                          chunk_overlap=0,
                                          separators=["\n\n"])

In [6]:
# 문서 전처리 함수 생성

def cleaning_docs(docs):
    docs = docs.load()
    lens = None
    for idx, doc in enumerate(docs):
        corpus = doc.page_content.replace("\xa0", "").replace("  ", " ").split("\n")
        if lens is None:
            lens = []
            for sentence in corpus:
                lens.append(len(sentence))
            length = sorted(lens)[len(lens)//2]
        else:
            pass

        cleaning_corpus = []
        for sentence in corpus[:-2]:
            if len(sentence) >= length:
                cleaning_corpus.append(sentence)
            else:
                cleaning_corpus.append(sentence+"\n\n")   
        docs[idx].page_content = "".join(cleaning_corpus)

    return docs

In [7]:
# 문서 전처리

hr_docs = cleaning_docs(hr_loader)
security_docs = cleaning_docs(security_loader)
onboard_docs = cleaning_docs(onboard_loader)
tools_docs = cleaning_docs(tools_loader)
culture_docs = cleaning_docs(culture_loader)

In [8]:
# 텍스트 스플리터를 이용한 문서 분할

hr_docs = splitter.split_documents(hr_docs)
security_docs = splitter.split_documents(security_docs)
onboard_docs = splitter.split_documents(onboard_docs)
tools_docs = splitter.split_documents(tools_docs)
culture_docs = splitter.split_documents(culture_docs)

In [9]:
# 문서 병합

total_docs = hr_docs+security_docs+onboard_docs+tools_docs+culture_docs

In [10]:
# 벡터스토어 생성

vector_store = FAISS.from_documents(embedding=embeddings, documents=total_docs)

In [11]:
# LLM 기반 Reranker

retriever = vector_store.as_retriever()
compressor = LLMChainExtractor.from_llm(llm)

In [12]:
reranked_retriever = ContextualCompressionRetriever(
    base_retriever=retriever,
    base_compressor=compressor
)

In [13]:
query = '연차는 몇일 사용할 수 있나요?'

In [14]:
docs = reranked_retriever.invoke(query)

In [15]:
docs

[Document(metadata={'producer': 'Skia/PDF m134', 'creator': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) HeadlessChrome/134.0.0.0 Safari/537.36', 'creationdate': '2025-06-20T10:32:30+00:00', 'title': '1. 인사 운영 메뉴얼', 'moddate': '2025-06-20T10:32:30+00:00', 'source': './files/rules/1._인사_운영_메뉴얼.pdf', 'total_pages': 5, 'page': 1, 'page_label': '2'}, page_content='연차휴가 : 법정 연차휴가 (1 년 15 일 , 이후 근속연수에 따라 가산 ) 를 부여합니다 . 연차는 별도의 승인 없이 자유롭게 사용 가능합니다 .')]

In [16]:
prompt = ChatPromptTemplate([
    ("system", "당신은 회사 내규 챗봇입니다. 사용자 정보와 회사 내규 문서가 주어집니다. 그것을 통해 사용자의 행동을 제시하세요.\n"
               "---"
               "문서 : {context}\n\n"
               "문서에서 응답을 찾을 수 없는 경우 '문서에서 응답을 찾을 수 없습니다.' 라고 답변하세요."), 
    ("user", "{query}")

])

In [17]:
def format_docs(docs):

    return "\n\n".join(doc.page_content for doc in docs)

In [18]:
chain = {
    "context": reranked_retriever | RunnableLambda(format_docs),
    "query": RunnablePassthrough()
} | prompt | llm

In [29]:
query = "어떤 복지가 있나요?"

In [30]:
result = chain.invoke(query)

In [31]:
print(result.content)

회사는 다음과 같은 복지 제도를 운영하고 있습니다: 

- 점심 식대 지원
- 간식 · 커피 무제한 제공
- 의료검진 지원
- 자기계발비 또는 도서구입비 지원
- 사내 동호회 지원

이러한 제도를 통해 직원들의 업무 만족도와 복지를 높이고 있습니다.


In [26]:
query = "원격근무는 언제 할 수 있나요?"

In [27]:
result = chain.invoke(query)

In [28]:
print(result.content)

원격근무는 주 2회까지 허용됩니다. 이를 통해 업무 효율과 워라밸을 지원합니다.


In [32]:
query = "퇴사를 계획중인데 어떻게 하면 되나요?"
result = chain.invoke(query)
print(result.content)

퇴사를 계획하고 계신다면, 다음 단계를 따라 주시기 바랍니다:

1. 최소 4주 이전에 퇴사 의사를 통보하는 것이 좋습니다. 이는 인수인계를 고려한 권장 사항입니다.
2. 퇴사 신청은 퇴직 의사 확인서를 제출하여 공식화해야 합니다.
3. 팀 리더 및 HR과 면담을 진행해야 합니다.

이 과정을 통해 퇴사를 원활하게 진행할 수 있습니다.


### 2-3. 점수 기반 Reranker

In [33]:
# 점수 기반 Reranker
model = CrossEncoder("BAAI/bge-reranker-base") 

In [34]:
def rerank(docs, top_n=5):
    pairs = [[query, doc.page_content] for doc in docs]
    scores = model.predict(pairs)
    reranked = sorted(zip(scores, docs), key=lambda x: x[0], reverse=True)
    result = [doc.page_content+f",  Score : {score}" for score, doc in reranked[:top_n]]
    return "\n".join(result)

In [35]:
query = "어떤 복지가 있나요?"

In [36]:
docs = retriever.invoke(query)

In [37]:
result = rerank(docs)

In [38]:
print(result)

급여 및 복리후생,  Score : 0.20158988237380981
위반 시 제재,  Score : 0.0006767080631107092


복지 제도 : 직원들의 업무 만족도와 복지를 위해 다음과 같은 제도를 운영합니다 : 점심 식대 지원 , 간식 · 커피 무제한 제공 , 의료검진 지원 , 자기계발비 또는 도서구입비 지원 , 사내 동호회 지원 등 . 스타트업 규모상 대기업만큼 풍부하지는 않지만 , 구성원의 업무 몰입과 성장에 도움이 되는 실용적인 복지를 추구합니다 .,  Score : 3.9664035284658894e-05
사이버 사고 대응,  Score : 3.7433277611853555e-05


In [39]:
query = "원격근무는 언제 할 수 있나요?"
docs = retriever.invoke(query)
result = rerank(docs)
print(result)



근무 형태 : 주 40시간제를 기본으로 하되 ,유연근무제를 도입하여 구성원이 자율적으로 근무 시간을 조정할 수 있습니다 . 코어타임 (10 시 ~16 시 ) 에 모두 근무하는 것을 원칙으로 ,  Score : 0.0004394112911541015


회의 , 외부 미팅 , 마감 등 모든 일정은 정시에 시작하고 끝내는 것을 기본으로 합니다 .유연근무제라도 합의된 시간에는 모두 온전히 참여해야 합니다 .약속을 어기게 될 경우 사전에 공유하고 대체 방안을 제시합니다 .규칙 2 . 호칭은 가볍게 , 태도는 진지하게 . ,  Score : 0.0001573135086800903
입사 첫 날,  Score : 3.7460595194716007e-05
하며 , 그 외 시간은 개인 일정에 맞게 조절 가능합니다 . 원격근무는 주 2 회까지 허용하여 업무 효율과 워라밸을 지원합니다 .,  Score : 3.7404621252790093e-05


In [40]:
query = "퇴사를 계획중인데 어떻게 하면 되나요?"
docs = retriever.invoke(query)
result = rerank(docs)
print(result)

퇴직 및 오프보딩,  Score : 0.2138497680425644


퇴사 통보 : 직원이 자발적으로 퇴사하고자 할 경우 , 인수인계를 고려하여 최소 4 주 이전에 퇴사 의사를 통보하는 것을 권장합니다 . 퇴사 신청은 퇴직 의사 확인서 제출을 통해 공식화되며 , 팀 리더 및 HR 과 면담을 거칩니다 .,  Score : 0.05010467767715454


연차는 자율적으로 사용하는 것이 원칙이며 , 팀 내 사전 공유만 잘 하면 됩니다 .정시 퇴근도 당연한 권리이며 , 휴가 중인 동료에게는 연락을 삼가는 문화를 지향합니다 .“퇴근해 ?ˮ , “ 휴가 좋겠네 ~ˮ 같은 농담도 서로 피합니다 .규칙 7 . 일을 시작할 땐 ‘ 왜 ʼ 부터 물어요 .,  Score : 0.0023380760103464127
입사 첫 날,  Score : 0.0012876279652118683


In [41]:
# chain 구성

chain = {
    "context": retriever | RunnableLambda(rerank),
    "query": RunnablePassthrough()
} | prompt | llm

In [42]:
query = "퇴사를 계획중인데 어떻게 하면 되나요?"
response = chain.invoke(query)
print(response.content)

퇴사를 계획하고 계시다면, 다음 단계를 따라 주시기 바랍니다:

1. **퇴사 통보**: 자발적으로 퇴사하고자 할 경우, 인수인계를 고려하여 최소 4주 이전에 퇴사 의사를 통보하는 것이 권장됩니다.

2. **퇴사 신청**: 퇴사 의사를 공식화하기 위해 퇴직 의사 확인서를 제출해야 합니다.

3. **면담**: 팀 리더 및 HR과 면담을 진행해야 합니다.

이 과정을 통해 퇴사를 원활하게 진행할 수 있습니다.


In [43]:
query = "원격근무는 언제 할 수 있나요?"
response = chain.invoke(query)
print(response.content)

원격근무는 주 2회까지 허용됩니다. 업무 효율과 워라밸을 지원하기 위해 원하시는 날에 맞춰 조정하실 수 있습니다.


In [44]:
query = "어떤 복지가 있나요?"
response = chain.invoke(query)
print(response.content)

회사는 직원들의 업무 만족도와 복지를 위해 다음과 같은 제도를 운영합니다:

- 점심 식대 지원
- 간식 및 커피 무제한 제공
- 의료검진 지원
- 자기계발비 또는 도서구입비 지원
- 사내 동호회 지원

스타트업 규모상 대기업만큼 풍부하지는 않지만, 구성원의 업무 몰입과 성장에 도움이 되는 실용적인 복지를 추구합니다.
